In [1]:
from segmentation import segment, init_segment_model
from augmentation import augment, init_augment_model
import SimpleITK as sitk
import os
import csv
import numpy as np
from util import XyzTuple, xyz2irc, logging, getCache
import collections
from collections import namedtuple  
import glob

In [2]:
CandidateInfoTuple = namedtuple('CandidateInfoTuple', 'isNodule_bool, hasAnnotation_bool, isMal_bool, diameter_mm, series_uid, center_xyz')
def getCandidateInfoList(requireOnDisk_bool=True):
    # We construct a set with all series_uids that are present on disk.
    # This will let us use the data, even if we haven't downloaded all of
    # the subsets yet.
    mhd_list = glob.glob('C:/Users/oplab/Desktop/Luna16_data/Luna16_img/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}

    candidateInfo_list = []
    with open('C:/Users/oplab/Desktop/Luna16_data/annotations_with_malignancy.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]
            annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm = float(row[4])
            isMal_bool = {'False': False, 'True': True}[row[5]] #it record the malignancy or not

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            candidateInfo_list.append(
                CandidateInfoTuple(
                    True,
                    True,
                    isMal_bool,
                    annotationDiameter_mm,
                    series_uid,
                    annotationCenter_xyz,
                )
            )

    with open('C:/Users/oplab/Desktop/Luna16_data/candidates_V2.csv', "r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid = row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue

            isNodule_bool = bool(int(row[4]))
            candidateCenter_xyz = tuple([float(x) for x in row[1:4]])

            if not isNodule_bool:
                candidateInfo_list.append(
                    CandidateInfoTuple(
                        False,
                        False,
                        False,
                        0.0,
                        series_uid,
                        candidateCenter_xyz,
                    )
                )

    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [3]:
def getCandidateInfoDict(requireOnDisk_bool=True):  #把candidateInfoList包成Dict
    candidateInfo_list = getCandidateInfoList(requireOnDisk_bool)
    candidateInfo_dict = {}

    for candidateInfo_tup in candidateInfo_list:
        candidateInfo_dict.setdefault(candidateInfo_tup.series_uid,
                                      []).append(candidateInfo_tup)

    return candidateInfo_dict

In [4]:
# candidate_dict =  getCandidateInfoDict()

In [5]:
# npy_path = glob.glob(os.path.join(write_root, sub_root, path) + ''.format(series_uid))

In [6]:
model_best, device = init_augment_model()
model, device = init_segment_model()

In [7]:
write_root_1 = "D:\\LUNA\\Luna16_SegData"
if not os.path.exists(write_root_1):
    os.mkdir("D:\\LUNA\\Luna16_SegData")
    
# read_root = "C:\\Users\\oplab\\Desktop\\Luna16_data\\Luna16_img"
read_root = "C:\\Users\\oplab\\Desktop\\Luna16_data\\Luna16_img"
    
append = True

# split to train test
for sub_root in os.listdir(read_root):
    root = os.path.join(read_root, sub_root)
    print(root)
    exist_folder = glob.glob(os.path.join(write_root_1, sub_root))
    if (sub_root != "subset0" and exist_folder != [] ):
        continue
    for path in os.listdir(root):
        if path.find("mhd") >= 0:
            raw_path_1 = os.path.join(write_root_1, sub_root, path)
            exist_path_1 =  glob.glob(os.path.join(write_root_1, sub_root, path))
            if (exist_path_1 != [] and append):
                print("pass")
                continue
            print(path)
            imagePath = os.path.join(root, path)
            ct_mhd = sitk.ReadImage(imagePath)
            if ct_mhd.GetDimension()==4 and ct_mhd.GetSize()[3]==1:
                ct_mhd = ct_mhd[...,0]
            origin = sitk.GetArrayFromImage(ct_mhd)
            
           # segment part
            segmented = segment(origin, model, device)
            hu_a = np.array(segmented, dtype=np.float32)
            # Load or create a SimpleITK image
            current_origin = ct_mhd.GetOrigin()
            current_spacing = ct_mhd.GetSpacing()

            sitk_image = sitk.GetImageFromArray(segmented)
            # image_short = sitk.Cast(image, sitk.sitkInt16)

            # Set the image origin, spacing, and direction (modify as needed)
            sitk_image.SetOrigin((current_origin[0], current_origin[1], current_origin[2]))
            sitk_image.SetSpacing((current_spacing[0], current_spacing[1], current_spacing[2]))

            # Save the image as a MetaImage file
            if not os.path.exists(os.path.join(write_root_1, sub_root)):
                os.mkdir(os.path.join(write_root_1, sub_root))
            print(os.path.join(write_root_1, sub_root, path))
            sitk.WriteImage(sitk_image, raw_path_1) 

C:\Users\oplab\Desktop\Luna16_data\Luna16_img\subset0
pass
pass
pass
pass
pass
pass
pass
pass
pass
1.3.6.1.4.1.14519.5.2.1.6279.6001.129055977637338639741695800950.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.129055977637338639741695800950.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.130438550890816550994739120843.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.130438550890816550994739120843.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.138080888843357047811238713686.mhd
D:\LUNA\Luna16_SegData\subset0\1.3.6.1.4.1.14519.5.2.1.6279.6001.138080888843357047811238713686.mhd
1.3.6.1.4.1.14519.5.2.1.6279.6001.1392587778987466933658